In [ ]:
import pandas as pd
import requests
import datetime
import math

# Global token map
token_df = pd.DataFrame()

def initializeSymbolTokenMap():
    global token_df
    url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
    d = requests.get(url).json()
    token_df = pd.DataFrame.from_dict(d)
    token_df["expiry"] = pd.to_datetime(token_df["expiry"], errors="coerce")
    token_df["strike"] = pd.to_numeric(token_df["strike"], errors="coerce")
    print("Symbol Token Map Initialized")

def getTokenInfo(exch_seg, instrumenttype, symbol, strike_price=None, pe_ce=None):
    global token_df
    df = token_df

    if exch_seg == "NSE" and instrumenttype == "EQ":
        return df[
            (df["exch_seg"] == "NSE") &
            (df["symbol"] == symbol)
        ]

    elif exch_seg == "NFO" and instrumenttype == "FUTIDX":
        return df[
            (df["exch_seg"] == "NFO") &
            (df["instrumenttype"] == instrumenttype) &
            (df["name"] == symbol)
        ]

    elif exch_seg == "NFO" and instrumenttype in ["OPTSTK", "OPTIDX"]:
        return df[
            (df["exch_seg"] == "NFO") &
            (df["instrumenttype"] == instrumenttype) &
            (df["name"] == symbol) &
            (df["strike"] == float(strike_price)) &
            (df["option_type"] == pe_ce)
        ]

    return pd.DataFrame()  # fallback

# Run logic
initializeSymbolTokenMap()

# Get BANKNIFTY token from NSE segment
nifty_token_info = getTokenInfo("NSE", "EQ", "BANKNIFTY")
if not nifty_token_info.empty:
    nifty_token = nifty_token_info.iloc[0]["token"]
    print("BANKNIFTY Token:", nifty_token)
else:
    print("BANKNIFTY token not found.")
